# Imports

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
import numpy as np
import math
from scipy.stats import norm


In [4]:
! pip install pandas-datareader==0.10.0

# Compiling metrics

## Glassnode



### Bitcoin On-chain Columns
#### Price/Market 
* Market Cap
* Realized Cap
* Delta Cap = Realized cap - Average cap

#### Transaction
* Exchange Net flow volume - difference of in volume flowing into exchanges and out of exchanges vs price
* Bitcoin: Number of Transactions per Second or count
Total number of daily transactions

#### Miner
* Miner Revenue - lesser miner revenue could discourage people to mine new cryptocurrency, therefore lower supply which increase cryptocurrency price.
* Percentage mine revenue: fees/minted coins

#### Profit and Loss
* Unrealized Profit/Loss
* Relative Unrealized profit or loss
* Net Unrealized Profit/Loss (NUPL) 
Fear index have a correlation with price

#### Stock index funds or bond interest rate
* SPY

#### Ratios
* Stock-to-flow 
* Stock-to-flow deflection
Is the stock undervalued or overvalued?

#### Indicators (https://cointelegraph.com/magazine/2021/09/03/how-to-prepare-for-end-of-bull-run-part-1-timing)
* Puell Multiple
* MVRV Z-Score

#### Volume
* Entity Balances
* Exchange Net Position Change - All Exchanges
supply held in exchange wallets

#### Unspent/Spent Outputs
* Percent of UTXOs in Profit

In [5]:
! unzip /content/btc.zip -d /content/dataset

Archive:  /content/btc.zip
   creating: /content/dataset/btc/indicators/
  inflating: /content/dataset/btc/indicators/mvrv-z-score.json  
  inflating: /content/dataset/btc/indicators/puell-multiple.json  
   creating: /content/dataset/btc/miner/
  inflating: /content/dataset/btc/miner/miner-revenue-fees.json  
  inflating: /content/dataset/btc/miner/miner-revenue-total.json  
   creating: /content/dataset/btc/price market/
  inflating: /content/dataset/btc/price market/btc-close-price.json  
  inflating: /content/dataset/btc/price market/delta-cap.json  
  inflating: /content/dataset/btc/price market/market-cap.json  
  inflating: /content/dataset/btc/price market/realized-cap.json  
   creating: /content/dataset/btc/profit loss/
  inflating: /content/dataset/btc/profit loss/net-unrealized-profit-loss-nupl.json  
  inflating: /content/dataset/btc/profit loss/relative-unrealized-loss.json  
  inflating: /content/dataset/btc/profit loss/relative-unrealized-profit.json  
   creating: /con

In [6]:
from functools import reduce

rootdir = '/content/'

#btc
dfs=[]

#price market
price = pd.read_json(rootdir+'dataset/btc/price market/btc-close-price.json')
price.rename({'v':'close'},axis=1,inplace=True)
dfs.append(price)

deltacap = pd.read_json(rootdir+'dataset/btc/price market/delta-cap.json')
deltacap.rename({'v':'deltacap'},axis=1,inplace=True)
dfs.append(deltacap)

# mcap = pd.read_json(rootdir+'dataset/btc/price market/market-cap.json')
# mcap.rename({'v':'marketcap'},axis=1,inplace=True)
# dfs.append(mcap)

rcap = pd.read_json(rootdir+'dataset/btc/price market/realized-cap.json')
rcap.rename({'v':'realizedcap'},axis=1,inplace=True)
dfs.append(rcap)

#miner
miner_rev_total = pd.read_json(rootdir+'dataset/btc/miner/miner-revenue-total.json')
miner_rev_total.rename({'v':'miner_revenue_total'},axis=1,inplace=True)
dfs.append(miner_rev_total)

miner_rev_fees = pd.read_json(rootdir+'dataset/btc/miner/miner-revenue-fees.json')
miner_rev_fees.rename({'v':'miner_fees_percent'},axis=1,inplace=True)
dfs.append(miner_rev_fees)

#profit loss
nupl = pd.read_json(rootdir+'dataset/btc/profit loss/net-unrealized-profit-loss-nupl.json')
nupl.rename({'v':'nupl'},axis=1,inplace=True)
dfs.append(nupl)

rup = pd.read_json(rootdir+'dataset/btc/profit loss/relative-unrealized-profit.json')
rup.rename({'v':'relative_unrealised_profit'},axis=1,inplace=True)
dfs.append(rup)

rul = pd.read_json(rootdir+'dataset/btc/profit loss/relative-unrealized-loss.json')
rul.rename({'v':'relative_unrealised_loss'},axis=1,inplace=True)
dfs.append(rul)

#transactions
net_flow_vol = pd.read_json(rootdir+'dataset/btc/transaction/exchange-net-flow-volume.json')
net_flow_vol.rename({'v':'net_flow_vol'},axis=1,inplace=True)
dfs.append(net_flow_vol)

tx_rate = pd.read_json(rootdir+'dataset/btc/transaction/transaction-rate.json')
tx_rate.rename({'v':'transaction_rate'},axis=1,inplace=True)
dfs.append(tx_rate)

tx_count = pd.read_json(rootdir+'dataset/btc/transaction/transaction-count.json')
tx_count.rename({'v':'total_transaction'},axis=1,inplace=True)
dfs.append(tx_count)

inhse_vol = pd.read_json(rootdir+'dataset/btc/transaction/in-house-exchange-volume.json')
inhse_vol.rename({'v':'in_house_exchange_volume'},axis=1,inplace=True)
dfs.append(inhse_vol)

#Ratio
rhodl = pd.read_json(rootdir+'dataset/btc/ratio/rhodl-ratio.json')
rhodl.rename({'v':'rhodl_ratio'},axis=1,inplace=True)
dfs.append(rhodl)

stfr = pd.read_json(rootdir+'dataset/btc/ratio/stock-to-flow-ratio.json')
stfr_inner = pd.io.json.json_normalize(stfr.o)
stfr['days_till_halving'] = stfr_inner['daysTillHalving']
stfr['stock_to_flow_ratio'] = stfr_inner['ratio']
del stfr['o']
dfs.append(stfr)

stfrd = pd.read_json(rootdir+'dataset/btc/ratio/stock-to-flow-deflection.json')
stfrd.rename({'v':'stock_to_flow_deflection'},axis=1,inplace=True)
dfs.append(stfrd)

#Indicators
puell = pd.read_json(rootdir+'dataset/btc/indicators/puell-multiple.json')
puell.rename({'v':'puell_multiple'},axis=1,inplace=True)
dfs.append(puell)

mvrv_z = pd.read_json(rootdir+'dataset/btc/indicators/mvrv-z-score.json')
mvrv_z.rename({'v':'mvrv_z_score'},axis=1,inplace=True)
dfs.append(mvrv_z)

#Volume
net_positions = pd.read_json(rootdir+'dataset/btc/volume/exchange-net-position-change.json')
net_positions.rename({'v':'exchange_net_positions_change'},axis=1,inplace=True)
dfs.append(net_positions)

#Unspent spent outputs
utxo = pd.read_json(rootdir+'dataset/btc/unspent spent outputs/percent-utx-os-in-profit.json')
utxo.rename({'v':'percent_utxos_in_profit'},axis=1,inplace=True)
dfs.append(utxo)

btc_final = reduce(lambda left,right: pd.merge(left,right,on='t'), dfs)

btc_final.rename({'t':'Date'},axis=1,inplace=True)
btc_final['Date'] = pd.to_datetime(btc_final['Date'])
btc_final.set_index('Date',inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [7]:
btc_final.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3285 entries, 2011-09-16 00:00:00+00:00 to 2020-09-12 00:00:00+00:00
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   close                          3285 non-null   float64
 1   deltacap                       3285 non-null   float64
 2   realizedcap                    3285 non-null   float64
 3   miner_revenue_total            3285 non-null   float64
 4   miner_fees_percent             3285 non-null   float64
 5   nupl                           3285 non-null   float64
 6   relative_unrealised_profit     3285 non-null   float64
 7   relative_unrealised_loss       3285 non-null   float64
 8   net_flow_vol                   3285 non-null   float64
 9   transaction_rate               3285 non-null   float64
 10  total_transaction              3285 non-null   int64  
 11  in_house_exchange_volume       3285 non-null   float64
 12  


### ETH On-chain Columns
#### Price/Market 
* Market Cap
* Realized Cap
* Delta Cap = Realized cap - Average cap

#### Transaction
* Exchange Net flow volume - difference of in volume flowing into exchanges and out of exchanges vs price
* Bitcoin: Number of Transactions per Second or count
Total number of daily transactions

#### Miner
* Miner Revenue - lesser miner revenue could discourage people to mine new cryptocurrency, therefore lower supply which increase cryptocurrency price.
* Percentage mine revenue: fees/minted coins

#### Profit and Loss
* Unrealized Profit/Loss
* Relative Unrealized profit or loss
* Net Unrealized Profit/Loss (NUPL) 
Fear index have a correlation with price

#### Stock index funds or bond interest rate
* SPY

#### Indicators (https://cointelegraph.com/magazine/2021/09/03/how-to-prepare-for-end-of-bull-run-part-1-timing)
* Puell Multiple
* MVRV Z-Score

#### Volume
* Entity Balances
* Exchange Net Position Change - All Exchanges
supply held in exchange wallets

#### Unspent/Spent Outputs
* Percent of UTXOs in Profit

In [8]:
! unzip /content/eth.zip -d /content/dataset

Archive:  /content/eth.zip
   creating: /content/dataset/eth/indicators/
  inflating: /content/dataset/eth/indicators/mvrv-z-score.json  
   creating: /content/dataset/eth/miner/
  inflating: /content/dataset/eth/miner/miner-revenue-fees.json  
  inflating: /content/dataset/eth/miner/miner-revenue-total.json  
   creating: /content/dataset/eth/price market/
  inflating: /content/dataset/eth/price market/eth-close-price.json  
  inflating: /content/dataset/eth/price market/market-cap.json  
  inflating: /content/dataset/eth/price market/realized-cap.json  
   creating: /content/dataset/eth/profit loss/
  inflating: /content/dataset/eth/profit loss/net-unrealized-profit-loss-nupl.json  
  inflating: /content/dataset/eth/profit loss/relative-unrealized-loss.json  
  inflating: /content/dataset/eth/profit loss/relative-unrealized-profit.json  
   creating: /content/dataset/eth/transaction/
  inflating: /content/dataset/eth/transaction/exchange-netflow-volume.json  
  inflating: /content/da

In [9]:
from functools import reduce

rootdir = '/content/'

#eth
dfs=[]

#price market
price = pd.read_json(rootdir+'dataset/eth/price market/eth-close-price.json')
price.rename({'v':'close'},axis=1,inplace=True)
dfs.append(price)

# mcap = pd.read_json(rootdir+'dataset/eth/price market/market-cap.json')
# mcap.rename({'v':'marketcap'},axis=1,inplace=True)
# dfs.append(mcap)

rcap = pd.read_json(rootdir+'dataset/eth/price market/realized-cap.json')
rcap.rename({'v':'realizedcap'},axis=1,inplace=True)
dfs.append(rcap)

#miner
miner_rev_total = pd.read_json(rootdir+'dataset/eth/miner/miner-revenue-total.json')
miner_rev_total.rename({'v':'miner_revenue_total'},axis=1,inplace=True)
dfs.append(miner_rev_total)

miner_rev_fees = pd.read_json(rootdir+'dataset/eth/miner/miner-revenue-fees.json')
miner_rev_fees.rename({'v':'miner_fees_percent'},axis=1,inplace=True)
dfs.append(miner_rev_fees)

#profit loss
nupl = pd.read_json(rootdir+'dataset/eth/profit loss/net-unrealized-profit-loss-nupl.json')
nupl.rename({'v':'nupl'},axis=1,inplace=True)
dfs.append(nupl)

rup = pd.read_json(rootdir+'dataset/eth/profit loss/relative-unrealized-profit.json')
rup.rename({'v':'relative_unrealised_profit'},axis=1,inplace=True)
dfs.append(rup)

rul = pd.read_json(rootdir+'dataset/eth/profit loss/relative-unrealized-loss.json')
rul.rename({'v':'relative_unrealised_loss'},axis=1,inplace=True)
dfs.append(rul)

#transactions
net_flow_vol = pd.read_json(rootdir+'dataset/eth/transaction/exchange-netflow-volume.json')
net_flow_vol.rename({'v':'net_flow_vol'},axis=1,inplace=True)
dfs.append(net_flow_vol)

tx_rate = pd.read_json(rootdir+'dataset/eth/transaction/transaction-rate.json')
tx_rate.rename({'v':'transaction_rate'},axis=1,inplace=True)
dfs.append(tx_rate)

tx_count = pd.read_json(rootdir+'dataset/eth/transaction/transaction-count.json')
tx_count.rename({'v':'total_transaction'},axis=1,inplace=True)
dfs.append(tx_count)

inhse_vol = pd.read_json(rootdir+'dataset/eth/transaction/in-house-exchange-volume.json')
inhse_vol.rename({'v':'in_house_exchange_volume'},axis=1,inplace=True)
dfs.append(inhse_vol)

#Indicators
mvrv_z = pd.read_json(rootdir+'dataset/eth/indicators/mvrv-z-score.json')
mvrv_z.rename({'v':'mvrv_z_score'},axis=1,inplace=True)
dfs.append(mvrv_z)

#Volume
net_positions = pd.read_json(rootdir+'dataset/eth/volume/exchange-net-position-change.json')
net_positions.rename({'v':'exchange_net_positions_change'},axis=1,inplace=True)
dfs.append(net_positions)

eth_final = reduce(lambda left,right: pd.merge(left,right,on='t'), dfs)

eth_final.rename({'t':'Date'},axis=1,inplace=True)
eth_final['Date'] = pd.to_datetime(eth_final['Date'])
eth_final.set_index('Date',inplace=True)

In [10]:
eth_final.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1832 entries, 2015-09-06 00:00:00+00:00 to 2020-09-10 00:00:00+00:00
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   close                          1828 non-null   float64
 1   realizedcap                    1832 non-null   float64
 2   miner_revenue_total            1832 non-null   float64
 3   miner_fees_percent             1832 non-null   float64
 4   nupl                           1832 non-null   float64
 5   relative_unrealised_profit     1832 non-null   float64
 6   relative_unrealised_loss       1832 non-null   float64
 7   net_flow_vol                   1832 non-null   float64
 8   transaction_rate               1832 non-null   float64
 9   total_transaction              1832 non-null   int64  
 10  in_house_exchange_volume       1832 non-null   float64
 11  mvrv_z_score                   1832 non-null   float64
 12  

In [11]:
eth_final[eth_final.isnull().any(axis=1)]

,close,realizedcap,miner_revenue_total,miner_fees_percent,nupl,relative_unrealised_profit,relative_unrealised_loss,net_flow_vol,transaction_rate,total_transaction,in_house_exchange_volume,mvrv_z_score,exchange_net_positions_change
Date,,,,,,,,,,,,,
2018-04-12 00:00:00+00:00,NaN,3.260675e+10,20577.595340,0.018412,0.273578,0.531837,0.258259,-21164.874861,7.836123,677041,141436.340386,0.326801,679666.133872
2018-04-13 00:00:00+00:00,NaN,3.256329e+10,21075.428770,0.020985,0.274772,0.531529,0.256756,-79046.285699,8.256644,713374,127761.029014,0.328377,584197.152921
2018-04-14 00:00:00+00:00,NaN,3.254232e+10,20703.428550,0.015313,0.275677,0.531572,0.255894,-13254.282148,6.903391,596453,49441.705835,0.329540,585003.245547
2018-04-15 00:00:00+00:00,NaN,3.252987e+10,20690.112531,0.014176,0.276029,0.531217,0.255188,-42203.790760,6.998171,604642,77453.379492,0.330020,515865.036165


In [12]:
from pandas_datareader import data
#imputing missing close price from data reader

null_start = eth_final[eth_final.isnull().any(axis=1)].index.min()
null_end = eth_final[eth_final.isnull().any(axis=1)].index.max()

eth_panel = data.DataReader(['ETH-USD'],'yahoo',null_start,null_end)
eth_panel.index= pd.to_datetime(eth_panel.index,utc=True)
eth_close = eth_panel['Close']['ETH-USD'].rename('close')
eth_close

Date
2018-04-11 00:00:00+00:00    492.941010
2018-04-12 00:00:00+00:00    492.734985
2018-04-13 00:00:00+00:00    501.477997
2018-04-14 00:00:00+00:00    531.702026
2018-04-15 00:00:00+00:00    511.147003
Name: close, dtype: float64

In [13]:
eth_final.loc[null_start:null_end,'close'] = eth_close

eth_final.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1832 entries, 2015-09-06 00:00:00+00:00 to 2020-09-10 00:00:00+00:00
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   close                          1832 non-null   float64
 1   realizedcap                    1832 non-null   float64
 2   miner_revenue_total            1832 non-null   float64
 3   miner_fees_percent             1832 non-null   float64
 4   nupl                           1832 non-null   float64
 5   relative_unrealised_profit     1832 non-null   float64
 6   relative_unrealised_loss       1832 non-null   float64
 7   net_flow_vol                   1832 non-null   float64
 8   transaction_rate               1832 non-null   float64
 9   total_transaction              1832 non-null   int64  
 10  in_house_exchange_volume       1832 non-null   float64
 11  mvrv_z_score                   1832 non-null   float64
 12  

In [14]:
# eth_final.iloc[949:953,0]
eth_final.loc[null_start:null_end,'close'] 

Date
2018-04-12 00:00:00+00:00    492.734985
2018-04-13 00:00:00+00:00    501.477997
2018-04-14 00:00:00+00:00    531.702026
2018-04-15 00:00:00+00:00    511.147003
Name: close, dtype: float64


### LTC On-chain Columns
#### Price/Market 
* Market Cap
* Realized Cap
* Delta Cap = Realized cap - Average cap

#### Transaction
* Exchange Net flow volume - difference of in volume flowing into exchanges and out of exchanges vs price
* Bitcoin: Number of Transactions per Second or count
Total number of daily transactions

#### Miner
* Miner Revenue - lesser miner revenue could discourage people to mine new cryptocurrency, therefore lower supply which increase cryptocurrency price.
* Percentage mine revenue: fees/minted coins

#### Profit and Loss
* Unrealized Profit/Loss
* Relative Unrealized profit or loss
* Net Unrealized Profit/Loss (NUPL) 
Fear index have a correlation with price

#### Stock index funds or bond interest rate
* SPY

#### Indicators (https://cointelegraph.com/magazine/2021/09/03/how-to-prepare-for-end-of-bull-run-part-1-timing)
* Puell Multiple
* MVRV Z-Score

#### Volume
* Entity Balances
* Exchange Net Position Change - All Exchanges
supply held in exchange wallets

#### Unspent/Spent Outputs
* Percent of UTXOs in Profit

In [15]:
! unzip /content/ltc.zip -d /content/dataset

Archive:  /content/ltc.zip
   creating: /content/dataset/ltc/indicators/
  inflating: /content/dataset/ltc/indicators/mvrv-z-score.json  
  inflating: /content/dataset/ltc/indicators/puell-multiple.json  
   creating: /content/dataset/ltc/price market/
  inflating: /content/dataset/ltc/price market/ltc-close-price.json  
  inflating: /content/dataset/ltc/price market/market-cap.json  
  inflating: /content/dataset/ltc/price market/realized-cap.json  
   creating: /content/dataset/ltc/transaction/
  inflating: /content/dataset/ltc/transaction/transaction-count.json  
  inflating: /content/dataset/ltc/transaction/transaction-rate.json  


In [16]:
from functools import reduce

rootdir = '/content/'

#ltc
dfs=[]

#price market
price = pd.read_json(rootdir+'dataset/ltc/price market/ltc-close-price.json')
price.rename({'v':'close'},axis=1,inplace=True)
dfs.append(price)

# mcap = pd.read_json(rootdir+'dataset/ltc/price market/market-cap.json')
# mcap.rename({'v':'marketcap'},axis=1,inplace=True)
# dfs.append(mcap)

rcap = pd.read_json(rootdir+'dataset/ltc/price market/realized-cap.json')
rcap.rename({'v':'realizedcap'},axis=1,inplace=True)
dfs.append(rcap)

#transactions
tx_rate = pd.read_json(rootdir+'dataset/ltc/transaction/transaction-rate.json')
tx_rate.rename({'v':'transaction_rate'},axis=1,inplace=True)
dfs.append(tx_rate)

tx_count = pd.read_json(rootdir+'dataset/ltc/transaction/transaction-count.json')
tx_count.rename({'v':'total_transaction'},axis=1,inplace=True)
dfs.append(tx_count)

# inhse_vol = pd.read_json(rootdir+'dataset/ltc/transaction/in-house-exchange-volume.json')
# inhse_vol.rename({'v':'in_house_exchange_volume'},axis=1,inplace=True)
# dfs.append(inhse_vol)

#Indicators
mvrv_z = pd.read_json(rootdir+'dataset/ltc/indicators/mvrv-z-score.json')
mvrv_z.rename({'v':'mvrv_z_score'},axis=1,inplace=True)
dfs.append(mvrv_z)

mvrv_z = pd.read_json(rootdir+'dataset/ltc/indicators/puell-multiple.json')
mvrv_z.rename({'v':'puell-multiple'},axis=1,inplace=True)
dfs.append(mvrv_z)

ltc_final = reduce(lambda left,right: pd.merge(left,right,on='t'), dfs)

ltc_final.rename({'t':'Date'},axis=1,inplace=True)
ltc_final['Date'] = pd.to_datetime(ltc_final['Date'])
ltc_final.set_index('Date',inplace=True)

In [17]:
ltc_final.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2308 entries, 2014-05-18 00:00:00+00:00 to 2020-09-10 00:00:00+00:00
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   close              2300 non-null   float64
 1   realizedcap        2308 non-null   float64
 2   transaction_rate   2308 non-null   float64
 3   total_transaction  2308 non-null   int64  
 4   mvrv_z_score       2308 non-null   float64
 5   puell-multiple     2308 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 126.2 KB


In [18]:
ltc_final[ltc_final.isnull().any(axis=1)]

,close,realizedcap,transaction_rate,total_transaction,mvrv_z_score,puell-multiple
Date,,,,,,
2014-11-09 00:00:00+00:00,NaN,3.949772e+08,0.031979,2763,-1.587256,0.280852
2014-11-10 00:00:00+00:00,NaN,3.949802e+08,0.041921,3622,-1.587361,0.270762
2014-11-11 00:00:00+00:00,NaN,3.950324e+08,0.035266,3047,-1.587758,0.297808
2018-04-12 00:00:00+00:00,NaN,7.075542e+09,0.377535,32619,-0.245356,1.202118
2018-04-13 00:00:00+00:00,NaN,7.071304e+09,0.378021,32661,-0.242876,1.134421
2018-04-14 00:00:00+00:00,NaN,7.070300e+09,0.284236,24558,-0.241648,1.142876
2018-04-15 00:00:00+00:00,NaN,7.053507e+09,0.294919,25481,-0.234237,1.215013
2019-03-31 00:00:00+00:00,NaN,4.499517e+09,0.262396,22671,-0.286329,0.895890


In [19]:
from pandas_datareader import data
#imputing missing close price from data reader

null_start = ltc_final[ltc_final.isnull().any(axis=1)].index.min()
null_end = ltc_final[ltc_final.isnull().any(axis=1)].index.max()
null_dates = ltc_final[ltc_final.isnull().any(axis=1)].index

ltc_panel = data.DataReader(['LTC-USD'],'yahoo',null_start,null_end)
ltc_panel.index= pd.to_datetime(ltc_panel.index,utc=True)
ltc_panel = ltc_panel['Close']['LTC-USD'].rename('close')
ltc_close = ltc_panel[null_dates][:8]
ltc_close

Date
2014-11-09 00:00:00+00:00      3.631460
2014-11-10 00:00:00+00:00      3.695630
2014-11-11 00:00:00+00:00      3.675890
2018-04-12 00:00:00+00:00    125.353996
2018-04-13 00:00:00+00:00    126.293999
2018-04-14 00:00:00+00:00    132.044998
2018-04-15 00:00:00+00:00    128.335007
2019-03-31 00:00:00+00:00     60.755398
Name: close, dtype: float64

In [20]:
ltc_final.update(ltc_close)
ltc_final.loc[null_dates]

,close,realizedcap,transaction_rate,total_transaction,mvrv_z_score,puell-multiple
Date,,,,,,
2014-11-09 00:00:00+00:00,3.631460,3.949772e+08,0.031979,2763,-1.587256,0.280852
2014-11-10 00:00:00+00:00,3.695630,3.949802e+08,0.041921,3622,-1.587361,0.270762
2014-11-11 00:00:00+00:00,3.675890,3.950324e+08,0.035266,3047,-1.587758,0.297808
2018-04-12 00:00:00+00:00,125.353996,7.075542e+09,0.377535,32619,-0.245356,1.202118
2018-04-13 00:00:00+00:00,126.293999,7.071304e+09,0.378021,32661,-0.242876,1.134421
2018-04-14 00:00:00+00:00,132.044998,7.070300e+09,0.284236,24558,-0.241648,1.142876
2018-04-15 00:00:00+00:00,128.335007,7.053507e+09,0.294919,25481,-0.234237,1.215013
2019-03-31 00:00:00+00:00,60.755398,4.499517e+09,0.262396,22671,-0.286329,0.895890


In [21]:
ltc_final.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2308 entries, 2014-05-18 00:00:00+00:00 to 2020-09-10 00:00:00+00:00
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   close              2308 non-null   float64
 1   realizedcap        2308 non-null   float64
 2   transaction_rate   2308 non-null   float64
 3   total_transaction  2308 non-null   int64  
 4   mvrv_z_score       2308 non-null   float64
 5   puell-multiple     2308 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 206.2 KB


## External Data

In [22]:
from pandas_datareader import data
import pandas_datareader as pdr

dfs = [btc_final,eth_final,ltc_final]
name = ['btc','eth','ltc']
for i in range(len(dfs)):
  panel = data.DataReader(['SPY'],'yahoo',dfs[i].index.min(),dfs[i].index.max())
  panel.index= pd.to_datetime(panel.index,utc=True)
  # panel.info()
  left = dfs[i]
  right = panel['Close']['SPY']
  df_final = pd.merge(left,right,on='Date')

  df_final.to_csv(f'{name[i]}_metrics_raw.csv')